In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from fastai.tabular.all import * 
from tabnet.model import TabNet

# Creating a Data Loader

In [3]:
data_dir = Path('./data')

In [4]:
def extract_gzip(file, dest=None):
    dest = dest or Path(dest)
    with gzip.open(file, 'rb') as f_in:
        with open(dest / file.stem, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [5]:
forest_type_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
forest_path = untar_data(forest_type_url, dest=data_dir, extract_func=extract_gzip)

In [6]:
df = pd.read_csv(forest_path, nrows=1_000, header=None); df.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [7]:
df = df_shrink(df)

In [8]:
for c in df.columns: 
    print(c, df[c].nunique(), df[c].agg(['min', 'max']).to_list())

0 451 [2486, 3267]
1 307 [0, 359]
2 34 [1, 40]
3 169 [0, 997]
4 160 [-45, 245]
5 866 [67, 6890]
6 108 [115, 254]
7 76 [141, 254]
8 150 [28, 240]
9 871 [120, 6853]
10 1 [1, 1]
11 1 [0, 0]
12 1 [0, 0]
13 1 [0, 0]
14 1 [0, 0]
15 1 [0, 0]
16 1 [0, 0]
17 1 [0, 0]
18 1 [0, 0]
19 1 [0, 0]
20 1 [0, 0]
21 2 [0, 1]
22 2 [0, 1]
23 1 [0, 0]
24 1 [0, 0]
25 2 [0, 1]
26 1 [0, 0]
27 1 [0, 0]
28 1 [0, 0]
29 2 [0, 1]
30 1 [0, 0]
31 2 [0, 1]
32 2 [0, 1]
33 2 [0, 1]
34 1 [0, 0]
35 2 [0, 1]
36 2 [0, 1]
37 2 [0, 1]
38 1 [0, 0]
39 1 [0, 0]
40 1 [0, 0]
41 1 [0, 0]
42 2 [0, 1]
43 2 [0, 1]
44 1 [0, 0]
45 1 [0, 0]
46 1 [0, 0]
47 1 [0, 0]
48 1 [0, 0]
49 1 [0, 0]
50 1 [0, 0]
51 1 [0, 0]
52 2 [0, 1]
53 2 [0, 1]
54 3 [1, 5]


In [9]:
dls = TabularDataLoaders.from_df(df, path=data_dir, y_names=54, y_block=CategoryBlock,
    cat_names =  [],
    cont_names = df.columns.difference([54]).to_list(),
    procs = [Categorify, FillMissing, Normalize], bs=3)

In [10]:
learn = TabularLearner(dls, TabNet(nf=54, nt=12))

In [11]:
learn.lr_find()

KEK
KEK


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

## Help 

* Did you get a `RuntimeError: Trying to backward through the graph a second time" error? Read [Retain Graph=True Error](https://stackoverflow.com/questions/46774641/what-does-the-parameter-retain-graph-mean-in-the-variables-backward-method) - Why did you get it? 